In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings


modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\Users\peter\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\peter\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformers.json: 100%|██████████| 116/116 [00

In [3]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

Load Best Buy Data

In [8]:
import json
import pandas as pd
with open('../../data/BestBuyWorkers.json', 'r') as f:
    workers = json.load(f)

In [16]:
def format_reddit_entry(entry: dict) -> str:
    title = entry['reddit_title'].strip() if entry['reddit_title'] else ''
    text = entry['reddit_text'].strip() if entry['reddit_text'] else ''
    text = text.replace('\n', ' ')
    if len(title)>0:
        if len(text)>0:
            return title+'; '+text
        return title
    else:
        if len(text)>0:
            return text
    return None

In [17]:
best_buy = pd.DataFrame(workers)
best_buy['sentence'] = [format_reddit_entry(worker) for worker in workers]
best_buy = best_buy.dropna(subset=['sentence'])
best_buy

,aware_post_type,aware_created_ts,reddit_id,reddit_name,reddit_created_utc,reddit_author,reddit_text,reddit_permalink,reddit_title,reddit_url,reddit_subreddit,reddit_link_id,reddit_parent_id,reddit_submission,sentence
0,submission,2023-04-16T17:32:45,12opsul,t3_12opsul,1681680765,utaustinresearch,,/r/BestBuyWorkers/comments/12opsul/research_st...,Research Study Recruitment - Managers,https://i.redd.it/nrmkf51rebua1.png,BestBuyWorkers,None,None,None,Research Study Recruitment - Managers
1,comment,2023-04-16T18:48:06,jgjgy9e,t1_jgjgy9e,1681685286,Not_A_Real_Boy69,![gif](giphy|YmQLj2KxaNz58g7Ofg)\n\n$50?,/r/BestBuyWorkers/comments/12opsul/research_st...,None,None,BestBuyWorkers,t3_12opsul,t3_12opsul,12opsul,![gif](giphy|YmQLj2KxaNz58g7Ofg) $50?
2,comment,2023-04-16T19:55:39,jgjpqmp,t1_jgjpqmp,1681689339,GSAgentsLivesMatter,bullshit on getting $50 its just a coupon to B...,/r/BestBuyWorkers/comments/12opsul/research_st...,None,None,BestBuyWorkers,t3_12opsul,t3_12opsul,12opsul,bullshit on getting $50 its just a coupon to B...
3,submission,2023-04-14T11:18:57,12m0ozl,t3_12m0ozl,1681485537,None,Good luck y’all,/r/BestBuyWorkers/comments/12m0ozl/sr_role_eli...,Sr role eliminated,https://www.reddit.com/r/BestBuyWorkers/commen...,BestBuyWorkers,None,None,None,Sr role eliminated; Good luck y’all
4,comment,2023-04-14T14:52:47,jg9its6,t1_jg9its6,1681498367,JiminyWillikerz,I was there almost 5 years before my position ...,/r/BestBuyWorkers/comments/12m0ozl/sr_role_eli...,None,None,BestBuyWorkers,t3_12m0ozl,t3_12m0ozl,12m0ozl,I was there almost 5 years before my position ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5624,comment,2024-03-07T16:11:30,kttl60a,t1_kttl60a,1709845890,Dquinones97,You do realize you’re asking for a bonus while...,/r/BestBuyWorkers/comments/1b91wdi/sti_bonus/k...,None,None,BestBuyWorkers,t3_1b91wdi,t3_1b91wdi,1b91wdi,You do realize you’re asking for a bonus while...
5625,comment,2024-03-07T16:18:42,kttmfik,t1_kttmfik,1709846322,Lucky-Beautiful6993,HERE YOUR BONUS🥒,/r/BestBuyWorkers/comments/1b91wdi/sti_bonus/k...,None,None,BestBuyWorkers,t3_1b91wdi,t3_1b91wdi,1b91wdi,HERE YOUR BONUS🥒
5626,comment,2024-03-07T15:59:47,kttj3mt,t1_kttj3mt,1709845187,Retro_Sphynx,What has leadership done to deserve a bonus?,/r/BestBuyWorkers/comments/1b91wdi/sti_bonus/k...,None,None,BestBuyWorkers,t3_1b91wdi,t3_1b91wdi,1b91wdi,What has leadership done to deserve a bonus?
5627,comment,2024-03-07T14:20:49,ktt37u3,t1_ktt37u3,1709839249,One-Negotiation7444,Board meeting finished this week. It’s always...,/r/BestBuyWorkers/comments/1b91wdi/sti_bonus/k...,None,None,BestBuyWorkers,t3_1b91wdi,t3_1b91wdi,1b91wdi,Board meeting finished this week. It’s always...


Fill Qdrant Database

In [18]:
# Create collection to store books
qdrant.create_collection(
    collection_name="best-buy",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [19]:
# vectorize!
# Note that for Coursera we use an older way of Qdrant doing the uploads using Records instead of Points
data = best_buy.to_dict(orient='records')

qdrant.upload_points(
    collection_name="best-buy",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["sentence"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable
    ]
)

In [20]:
user_query = 'What do employees of Best Buy think of the company?'

In [26]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="best-buy",
    query_vector=encoder.encode(user_query).tolist(),
    limit=20
)
for hit in hits:
  print(hit.payload['sentence']+"; score:", hit.score)

Best Buy used to be a great company to work for before Corie the CEO took over. Employees used to love working there, staffing was great, it wasn’t all about stupid sales metrics. Times have changed; score: 0.723883485187244
The fact we have employees asking on a subreddit how to do their jobs shows how bad Best Buy is doing; score: 0.7165790661809885
What is your opinion on Best Buy currently.; score: 0.7160576968713319
It honestly feels like Best Buy wants this type now.  Company used to be about taking care of the customers and its employees.  Now it's all about tricking them.  I don't like this place anymore.; score: 0.7092497934374563
Best Buy is gonna end up like circuit city in the next 5-10 years anyways. Her boss is probably the one micromanaging her about p&l’s and stupid shit like that. And of course, shit rolls down hill. Corporate makes regular jobs suck ass.; score: 0.6842804127788847
Current state of Best Buy; At this point no employee is safe; score: 0.6792654808907821
